# Генерация входного датасета

## Импорт библиотек

In [161]:
import random
import pandas as pd
from datetime import datetime, timedelta

## Счётчики данных

In [162]:
wrong_enter = 0
unplanned_enter = 0
planned_enter = 0

errors_amount : int

## Множества входных данных

In [163]:
full_names_52 = [
    "Семёнов Дмитрий Иванович",
    "Головин Сергей Вальерьевич",
    "Путилов Андрей Маркович",
    "Лапухов Алексей Дмитриевич",
    "Дружинин Георгий Михайлович",    
]

full_names_22 = [
    "Работаев Даниил Игоревич",
    "Работаева Ирина Генриховна"
]

full_names_dinner = [
    "Семёнов Дмитрий Иванович",
    "Головин Сергей Вальерьевич",
    "Работаев Даниил Игоревич",
    "Работаева Ирина Генриховна"
]

status = [
    "Вход",
    "Выход",
    "Доступ запрещён"
]

emergency_events = [
    "Учебная пожарная эвакуация",
    "Проверка систем голосового оповещения и сигнализации",
    "Учения по антитеррористической безопасности"
]

## Формирование отчётного периода

In [164]:
def GenerateReportingPeriods() -> tuple:
    start_date = datetime(
        year=2006,
        month=4,
        day=1,
        hour=8,
        minute=0,
        second=0
    )

    stop_date = start_date.replace(month=7, day=3)
    date_range = (stop_date - start_date).days
    
    return (
        [start_date + timedelta(days=day) 
            for day in range(date_range) 
            if (start_date + timedelta(days=day)).weekday() not in [5, 6]],
        [start_date + timedelta(days=day)
            for day in range(date_range)
            if (day % 4) < 2]
    )

## Утилиты для генерации

In [165]:
def GetProbability():
    return random.randint(0, 99) < 4

def GetTimeOffset(hours_max_offset, minutes_max_offset, seconds_max_offset) -> timedelta:
    return timedelta(
            hours=random.randint(0, hours_max_offset),
            minutes=random.randint(0, minutes_max_offset),
            seconds=random.randint(0, seconds_max_offset)
        ) 

### Генерация множества запланированных событий, подразумевающих выход не через турникет.

In [166]:
def GenerateEmergency(reporting_period : list) -> list:
    acc = []
    
    emergency_amounts = round(len(reporting_period) * 0.1)
    samples = random.sample(reporting_period, emergency_amounts)

    for current_date in samples:
        acc.append({
            "name" : random.choice(emergency_events),
            "event_dt" : current_date.replace(hour=11) + GetTimeOffset(1, 0, 0),
            "duration" : random.choice([10, 15, 20, 25, 30])  
        })

    return acc

## Внесение разнообразия в данные

In [167]:
def DiversifyEnter(name : str, current_dt : datetime) -> tuple:
    acc = []

    last_dt = current_dt + GetTimeOffset(2, 30, 59)

    #С вероятностью 5% сотрудник попытался войти слишком рано или через неправильный турникет
    if GetProbability():
        global wrong_enter
        wrong_enter += 1
        acc.append({
            "full_name" : name,
            "event_dt" : last_dt - GetTimeOffset(0, 9, 59) - timedelta(seconds=1),
            "status" : status[2]
        })

    #Входим
    acc.append({
        "full_name" : name,
        "event_dt" : last_dt,
        "status" : status[0]
    })

    return (acc, last_dt)


def DiversifyDay(name : str, last_dt : datetime) -> list:
    acc = []

    #Добавляем выход и вход тем, кто обедает за пределами офиса
    if name in full_names_dinner:
        last_dt += timedelta(hours=4, minutes=30) + GetTimeOffset(0, 30, 59)

        acc.append({
            "full_name" : name,
            "event_dt" : last_dt,
            "status" : status[1]
        })

        last_dt += timedelta(minutes=30) + GetTimeOffset(0, 30, 59)
        
        acc.append({
            "full_name" : name,
            "event_dt" : last_dt,
            "status" : status[0]
        })
    
    #С вероятностью 0.05 сотрудник покинул офис не через турникет и это не было запланированно
    if GetProbability():
        global unplanned_enter
        unplanned_enter += 1
        last_dt += timedelta(minutes=1) + GetTimeOffset(0, 20, 59)

        acc.append({
            "full_name" : name,
            "event_dt" : last_dt,
            "status" : status[0]
        }) 
        
    return acc

## Генерация данных

Опишем временные интервалы событий

Для графика 5/2:
| NAME | MIN | MAX | STATUS |
|:---|:---:|:---:|:---|
| Ошибочный вход | 07:50:00 | 10:30:58 | Доступ запрещён |
| Первый вход | 08:00:00 | 10:30:59 | Вход |
| Запланированный вход | 11:10:00 | 12:39:59 | Вход |
| Выход на обед | 13:00:59 | 15:31:58 | Выход |
| Вход с обеда | 13:30:59 | 16:32:57 | Вход |
| Незапланированный вход без обеда | 08:01:00 | 10:51:58 | Вход |
| Незапланированный вход с обедом | 13:31:59 | 16:53:56 | Вход |
| Нормальный выход | 17:00:00 | 22:01:58 | Выход |
| Выход после ночной смены | 04:00:00 | 07:01:58 | Выход |

In [168]:
def GenerateData(reporting_period_52 : list, reporting_period_22 : list, emergency : list) -> list:
    acc = []

    #График 5/2
    for current_date in reporting_period_52:
        for name in full_names_52:
            #Вносим разнообразие в данные
            tmp, last_dt = DiversifyEnter(name, current_date)
            acc += DiversifyDay(name, last_dt)
            acc += tmp

            #С вероятностью 0.05 сотрудник задерживается, присваиваем статус "Доступ запрещён"
            if GetProbability():
                working_hours = timedelta(hours=10, minutes=58, seconds=1)
                exit_status = status[2]
            else:
                working_hours = timedelta(hours=9)
                exit_status = status[1]

            #Уходим
            acc.append({
                "full_name" : name,
                "event_dt" : current_date + working_hours + GetTimeOffset(2, 30, 59),
                "status" : exit_status
            })

    #Грфик 2/2
    for i in range(0, len(reporting_period_22) - 1, 2):
        for name in full_names_22:
            first_date = reporting_period_22[i]
            second_date = reporting_period_22[i + 1]

            #Вносим разнообразие в данные
            tmp, last_dt = DiversifyEnter(name, first_date)
            acc += DiversifyDay(name, last_dt)
            acc += tmp

            #В первый день не уходим, второй день начинаем с обеда
            last_dt = second_date + GetTimeOffset(2, 30, 59)
            acc += DiversifyDay(name, last_dt)

            #Уходим во второй день
            acc.append({
                "full_name" : name,
                "event_dt" : last_dt + timedelta(hours=9) + GetTimeOffset(2, 30, 59),
                "status" : status[1]
            })

    #Обрабатываем непарный элемент (отчётный период кончается после первого дня)
    if len(reporting_period_22) % 2 != 0:
        current_date = reporting_period_22[-1]

        for name in full_names_22:
            #Вносим разнообразие в данные
            tmp, last_dt = DiversifyEnter(name, current_date)
            acc += DiversifyDay(name, last_dt)
            acc += tmp

    #Запланированные учения
    for elem in emergency:
        for name in full_names_52 + full_names_22:
            global planned_enter
            planned_enter += 1
            acc.append({
                "full_name" : name,
                "event_dt" : elem["event_dt"] + timedelta(minutes=elem["duration"]) + GetTimeOffset(0, 9, 59),
                "status" : status[0]
            })
    return acc

## Добавление ошибок

In [169]:
def GenerateErrors(data : pd.DataFrame) -> pd.DataFrame:
    global errors_amount
    errors_amount = round(len(data) * 0.05)
    samples = data.sample(errors_amount)
    copies = samples.copy()

    copies["event_dt"] += GetTimeOffset(0, 0, 58) + timedelta(seconds=1)

    return copies

## "Сборка" датасета

In [170]:
rp_52, rp_22 = GenerateReportingPeriods()
emergency = GenerateEmergency([i for i in rp_22 if i in rp_52])

data = GenerateData(rp_52, rp_22, emergency)

df_data = pd.DataFrame(data)
df_data = pd.concat([df_data, GenerateErrors(df_data)], ignore_index=True)
df_emergency = pd.DataFrame(emergency)

df_data = df_data.sort_values(by=["full_name", "event_dt"])

df_data.to_csv("data/source/entries_src.csv")
df_emergency.to_csv("data/reference/emergency_ref.csv")

## Количество вхождений различных данных в датасет

In [ ]:
print(f"Кол-во неуспешных входов: {wrong_enter}")
print(f"Кол-во незапланированных входов: {unplanned_enter}")
print(f"Кол-во запланированных входов: {planned_enter}")
print(f"Кол-во ошибок системы: {errors_amount}")
print((len(df_data) - wrong_enter - unplanned_enter - errors_amount + planned_enter) / 2)
print(len(rp_52) * 5)
print(len(rp_22) * 2)

Кол-во неуспешных входов: 17
Кол-во незапланированных входов: 19
Кол-во запланированных входов: 21
Кол-во ошибок системы: 62
617.0
325
